__FILE NAME FORMAT__:
{ImageType}\_{Filter}\_{ExposureTime}\_{FileNumber}.fits

__FILTER NAME FORMAT__:
- Red = R
- Blue = B
- Green = G
- Bessel R = BR
- Bessel B = BB
- Bessel G = BG
- None = CLEAR
- OIII = O3
- SII = S2
- Infrared = I
- Bessel Infrared = BI
- UV = UV
- Bessel UV = BUV
- H Beta = HB
- H Alpha = HA

### Configurations

In [165]:
# imports
from astropy.nddata import CCDData
from astropy.stats import mad_std
from astropy.utils.exceptions import AstropyWarning
from astropy import units
from pathlib import Path
from rich.logging import RichHandler

import ccdproc
import logging
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import warnings

In [166]:
# logging configurations
# suppresses the fits fixed warning (annoying)
warnings.filterwarnings("ignore", category=AstropyWarning, append=True)

# configure loggers
# outputs to terminal but will save any errors to a log
log = logging.Logger(name="DataReducerLog")
formatter = logging.Formatter("%(name)s|%(asctime)s|[%(levelname)s]|:%(message)s")
log.setLevel(logging.DEBUG)

stream_handler = RichHandler()
stream_handler.setLevel(logging.DEBUG) # change this to change terminal readout
file_handler = logging.FileHandler(filename="debug.log", delay=True)
file_handler.setFormatter(formatter)
file_handler.setLevel(logging.ERROR) # change this to change what is logged to file

log.addHandler(stream_handler)
log.addHandler(file_handler)

In [80]:
# configure file directies (for testing)
# TODO: Find better method/make this more configurable
# consider having this run from the raw data directory and just have the processed
# output nested within
raw_data_path = Path("RawData")
processed_data_path = Path("ProcessedData")
# ensure the processed data dir is there
os.makedirs(processed_data_path, exist_ok=True)

### Load Data

In [167]:
# load and sort data from raw data directory
raw_images = ccdproc.ImageFileCollection(raw_data_path)

log.info("Loaded {0} raw images".format(len(raw_images.files_filtered())))

### Prepare Functions

In [133]:
def exp_time_to_str(exposure_time):
    if int(exposure_time) == exposure_time:
        return str(int(exposure_time)) + "s"
    elif int(exposure_time*1000) == exposure_time*1000:
        return str(int(exposure_time*1000)) + "ms"

### Create Master Calibration Files

In [169]:
# create master bias
raw_biases = raw_images.files_filtered(IMTYPE="Bias", include_path=True)
master_filename = os.path.join(processed_data_path, "MasterBias.fits")

master_bias = ccdproc.combine(raw_biases, method="median", unit="adu")
master_bias.write(master_filename, overwrite=True)

log.info("Created Master Bias from {0} images -> {1}".format(len(raw_biases), master_filename))

# TODO: Remove/configure the overwrite function

In [170]:
# create master darks
dark_times = set([CCDData.read(t, unit="adu").meta.get("EXPTIME") for t in list(raw_images.files_filtered(IMTYPE="Dark", include_path=True))])
master_darks = {}

for time in dark_times:
    selected_darks = raw_images.files_filtered(EXPTIME=time, IMTYPE="Dark", include_path=True)
    save_time = exp_time_to_str(time)
    master_filename = os.path.join(processed_data_path, "MasterDark{0}s.fits".format(save_time))

    master_dark = ccdproc.combine(selected_darks, method="median", unit="adu")
    master_dark.write(master_filename, overwrite=True)

    master_darks[save_time] = CCDData.read(master_filename)

    log.info("Created {0} Master Dark from {1} images -> {2}".format(save_time, len(selected_darks), master_filename))

In [171]:
# create master flats
# list from the filter format above (WIP)
raw_flats = {f:[] for f in ["R", "B", "G", "BR", "BG", "BB", "I", "H3", "S2", "HA", "HB", "CLEAR"]}

# sort raw flats into the raw flats dict using above formatting (index 1 = filter)
for flat in list(raw_images.files_filtered(IMTYPE="Flat")) + list(raw_images.files_filtered(IMTYPE="Sky")):
    flat_filter = flat.split("_")[1].upper()

    if flat_filter in raw_flats:
        raw_flats[flat_filter].append(os.path.join(raw_data_path, flat))

# create master flats
master_flats = {}

for flat_filter, raw_list in raw_flats.items():
    # find all unique exposure times in the flat files
    flat_times = list(set(CCDData.read(os.path.join(t), unit="adu").meta.get("EXPTIME") for t in raw_list))

    # checks for uniformity in flat times and skips non-uniform flat collections
    # TODO: Handle this better (may not be neccesary but consider it)
    if len(flat_times) != 1:
        continue

    flat_time = flat_times[0]
    master_filename = os.path.join(processed_data_path, "MasterFlat{0}.fits".format(flat_filter))

    master_flat = ccdproc.combine(raw_list, method="median", unit="adu")
    master_flat = ccdproc.subtract_bias(master_flat, master_bias)
    master_flat = ccdproc.subtract_dark(master_flat, master_darks[exp_time_to_str(flat_time)], dark_exposure=flat_time * units.second, data_exposure=flat_time * units.second)
    master_flat.write(master_filename, overwrite=True)

    master_flats[flat_filter] = CCDData.read(master_filename)

    log.info("Created {0} {1} Master Flat from {2} images -> {3}".format(flat_filter, exp_time_to_str(flat_time), len(raw_list), master_filename))

# TODO: Fill out filter list

### Reduce Raw Lights

In [172]:
# reduce lights
# get all unique objects
objects_list = set([CCDData.read(l, unit="adu").meta.get("OBJECT") for l in raw_images.files_filtered(IMTYPE="Light", include_path=True)])

for obj_observed in objects_list:
    obj_name = obj_observed.strip().replace(" ", "")
    obj_save_dir = os.path.join(processed_data_path, obj_name)
    os.makedirs(obj_save_dir, exist_ok=True)

    for image in raw_images.files_filtered(IMTYPE="Light", OBJECT=obj_observed):
        ccd_image = CCDData.read(os.path.join(raw_data_path, image), unit="adu")
        reduced_filename = os.path.join(obj_save_dir, "Reduced_" + image)
        exposure_time = ccd_image.meta.get("EXPTIME")
        exposure_string = exp_time_to_str(exposure_time)
        image_filter = image.split("_")[1].upper()

        # ensure the exposure time has been processed
        if exposure_string not in master_darks:
            continue

        # ensure the filter has been processed
        if image_filter not in master_flats:
            continue

        reduced = ccdproc.subtract_bias(ccd_image, master_bias)
        reduced = ccdproc.subtract_dark(reduced, master_darks[exposure_string], dark_exposure=exposure_time * units.second, data_exposure=exposure_time * units.second)
        reduced = ccdproc.flat_correct(reduced, master_flats[image_filter])
        reduced.write(reduced_filename, overwrite=True)

        log.info("Reduced {0} using {1} Filter {2} Exposure {3} Object -> {4}".format(image, image_filter, exposure_string, obj_name, reduced_filename))

# TODO: Add logging throughout
        